In [257]:
from sklearn.datasets import load_iris,load_diabetes, load_breast_cancer
from sklearn.model_selection import train_test_split
from scipy import stats
from collections import Counter
import numpy as np 
from matplotlib import pyplot as plt
import math
import os 

### Intuition 
***
Intuitively a strong learner is a classification algorithm that can approximate the true solution up to a small error $\epsilon$. The goal of machine learning is to construct strong learners. However, they are often hard to construct and computationally expensive. A "Weak learner" is an algorithm that is just slightly better than random guessing: for a classification problem with balanced classes, its 0-1-loss is just a tiny bit better than random guessing: $50\% + \epsilon$ The idea of boosting is to combine many weak learners to obtain a strong learner 


### The outline
***
- Given a training set of $n$ points $(x_i,y_i)$ the boosting algorithm proceeds in the $t$ rounds.
- Training points have weights that change from round to round. The weights always add up to one.
- In each round, we train the weak classifier on the training points with the current weights. We then update the weights: 
    - For points $x_i$ that were misclassified, we increase their weight $w_i$ 
    - For points $x_i$ that got correctly classified we decrease their weights $w_i$
- At the very end the final classifier is a weighted sum, some kind of weighted majority vote, of the weak classifiers of each round.


### Some remarks 
***
- As opposed to random forest, which generates randomness through subsampling points and dimensions, the training set in boosting is always the same. We just change its weights.
- By re-weighting, the algorithm can focus on those examples that it finds difficult, or on the aspect that has been overlooked so far. 
- By the definition of a weak classifier, once a data point has accumulated weights larger than $50\%$ the weak algorithm will get it right. But it is not obvious that this helps the classifier because there are many points that we need right... So the question is how to combine all the evidence of the weak classifiers in such a way that we obtain a strong classifier at the end 


### AdaBoost algorithm
***
<strong>Input:</strong></br>
$\quad$ Training set: $\{S|s_{i} = (x_i, y_i) \forall i\}$ </br>
$\quad$ Weak Learner: $WL$ </br>
$\quad$ Number of rounds: $T$ </br>

<strong>Initialize:</strong></br>
$\quad$ $D^{(1)} = (\frac{1}{m},...,\frac{1}{m})$

<strong>for $t=1,...T$:</strong></br>
$\quad$ Invoke weak learner $h_{t} = WL(D^{(t)},S)$ </br>
$\quad$ Compute $\epsilon_{t} = \sum_{i=1}^{m}D^{(t)} \mathbb{1}_{[y_{i} \neq h_{t}(x_i)]}$ </br>
$\quad$ let $w_{t} = \frac{1}{2}log(\frac{1}{\epsilon_{t}} - 1)$ </br>
$\quad$ Update $D_{i}^{(1+t)} = \frac{ D_{i}^{(t)} \exp\left(-w_{t}y_{i}h_{t}(x_i)\right)}{\sum_{j=1}^{m}\left(D_{j}^{(t)}\exp(-w_{t}y_{j}h_{t}(x_j))\right)}$ for all $i=1,...,m$</br>

<strong>Output:</strong></br>
$\quad$ The hypothesis $h_{s}(x) = sign \left( \sum_{t=1}^{T}w_{t} h_{t}(x)\right)$


### Theoretical resoures
***
$(*)$ https://www.youtube.com/watch?v=4B5sp5Hw9Qc&list=PL05umP7R6ij2XCvrRzLokX6EoHWaGA2cC&index=28 </br>
$(*)$ https://en.wikipedia.org/wiki/AdaBoost </br>

In [175]:
X, Y, _, desc, feature_names, _, _ = load_breast_cancer().values()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [327]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None
        
                      
        
class DesicionTree:
    
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.weights = None
        self.root = None
        self.criteria = dict(
            gini = lambda Y,W : np.sum([np.log2(v/len(Y)) * v/len(Y) \
                                        for v,w in zip(Counter(Y).values(), W) ] / np.sum(W)),
            l2   = lambda x: x,
        )
    
    def fit(self, X, y, sample_size=1, weights=None):
        self.weights = np.array([1/len(y) for _ in range(len(y))]) if(weights is None) else weights 
        self.root = self._build_tree(X, y, sample_size)
        return self
        
    def _information_gain(self, n, total_loss, n_left, left_loss, n_rigth, right_loss):
        # Calculate information gain
        if n_left == 0 or n_rigth == 0: 
            return 0
        return total_loss - ((n_left / n) * left_loss + (n_rigth / n) * right_loss)
    
    def _create_split(self, data, split):
        # Split data based on split point
        left_idx = np.argwhere(data <= split).flatten()
        right_idx = np.argwhere(data > split).flatten()
        return left_idx, right_idx
        
    
    def _get_split(self, X, y, sample_size, criteria='gini'):
        score = []      
        total_loss = self.criteria[criteria](y, self.weights)
        features = np.random.choice(X.shape[1], size= math.ceil(X.shape[1] * sample_size), replace=False)
        for dimension, data in zip(features, X[:, features].T):
            for split in np.unique(data):
                # Get indices of each group given the split
                left_idx, right_idx = self._create_split(data, split)

                # Get loss of each group given the split
                left_loss = self.criteria[criteria](y[left_idx], self.weights)
                right_loss = self.criteria[criteria](y[right_idx], self.weights)
                
                # Calculate information gain
                ig = self._information_gain(len(y) ,total_loss, len(left_idx), left_loss, len(right_idx), right_loss)
                score.append((ig, dimension, split))   
                
        return min(score, key = lambda e: e[0])
    
    
    def _build_tree(self, X, y, sample_size, depth=0):
        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))
        
        (ig, dimension, split) = self._get_split(X, y, sample_size)
        left_idx, right_idx = self._create_split(X[:, dimension], split)
        
        # stopping criteria
        if (depth >= self.max_depth 
            or self.n_class_labels == 1 
            or self.n_samples < self.min_samples_split
            or len(left_idx) == 0 
            or len(right_idx) == 0      
           ):
            most_common_Label = Counter(y).most_common(1)[0][0]  
            return Node(value=most_common_Label)
        else:
            left_child = self._build_tree(X[left_idx, :], y[left_idx], sample_size, depth + 1)
            right_child = self._build_tree(X[right_idx, :], y[right_idx], sample_size, depth + 1)
            return Node(feature=dimension, threshold=split, left=left_child, right=right_child)

    def _traverse_tree(self, x, node):
        if node.is_leaf(): 
            return node.value  
        if x[node.feature] <= node.threshold: 
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
            
    def predict(self, X):
        predictions = [self._traverse_tree(x, self.root) for x in X]
        return np.array(predictions)
    
    
class AdaBoost:
    
    def __init__(self, max_depth=100, min_samples_split=2, size=10):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.weights = None
        self.epsilon = None
        self.size = size
        
    def fit(self, X, y):
        m = len(y)
        y = np.array([1 if e == 1 else -1 for e in y])
        
        self.trees = [DesicionTree(max_depth = 100) for _ in range(self.size)] 
        self.epsilon = np.zeros(self.size)
        
        w = np.array([ (1/m) for _ in range(m)])
        for t, tree in enumerate(self.trees):
            h = tree.fit(X,y,weights=w).predict(X)
            e = np.sum([weight for y_pred, y_truth, weight in zip(h,y,w) if y_pred == y_truth])         
            epsilon = (1/2) * np.log((1/e)-1) 
            self.epsilon[t] = epsilon
            w = (w * np.exp( -y * epsilon * h)) 
            w = w / np.sum(w)
        
    def predict(self, X):
        predictions = []
        for tree, epsilon in zip(self.trees, self.epsilon):
            predictions.append(tree.predict(X) * epsilon)
        f = np.sum( np.array(predictions), axis=0 )
        return(np.sign(f))
            
            

    



In [328]:
model = AdaBoost(size=2);
model.fit(X_train, y_train)
y_pred = model.predict(X_test)